# Dropout
Dropout[1] - метод регуляризации нейронных сетей путем случайной установки некоторых признаков в ноль во время прямого распространения. В этом задании необходимо реализовать слой Dropout и модифицировать полносвязанный класс, чтобы, при необходимости, использовать Dropout.

[1] [Geoffrey E. Hinton et al, "Improving neural networks by preventing co-adaptation of feature detectors", arXiv 2012](https://arxiv.org/abs/1207.0580)

In [ ]:
# Выполните начальные установки блокнота.
# Внимание! Блокнот работает в версии Python 3.6!

from __future__ import print_function
import time
import numpy as np
import matplotlib.pyplot as plt
from dlcv.classifiers.fc_net import *
from dlcv.data_utils import get_CIFAR10_data
from dlcv.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from dlcv.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # установка размеров графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Для перезагрузки внешних модулей python;
# см. http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """  возвращает относительную ошибку  """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# Загрузка и предобработка данных CIFAR10
data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

# Dropout: прямой путь
В файле `dlcv / layers.py`, реализуйте прямое распространение для Dropout. Поскольку Dropout ведет себя по-разному во время обучения и тестирования, обязательно реализуйте операцию для обоих режимов.

Затем запустите ячейку ниже, чтобы проверить реализацию (возможно Вам повезло и код уже реализован).

In [ ]:
np.random.seed(231)
x = np.random.randn(500, 500) + 10

for p in [0.25, 0.4, 0.7]:
  out, _ = dropout_forward(x, {'mode': 'train', 'p': p})
  out_test, _ = dropout_forward(x, {'mode': 'test', 'p': p})

  print('Выполнение тестов с  p = ', p)
  print('Среднее входа: ', x.mean())
  print('Средний выход в режиме обучения: ', out.mean())
  print('Средний выход в режиме тестирования: ', out_test.mean())
  print('Доля обнуленных выходов в режиме обучения: ', (out == 0).mean())
  print('Доля обнуленных выходов в режиме тестирования: ', (out_test == 0).mean())
  print()

# Dropout : обратное распространение
В файле `dlcv / layers.py`, реализуйте обратное распространение для Dropout. После этого запустите код ниже для проверки градиента.

In [ ]:
np.random.seed(231)
x = np.random.randn(10, 10) + 10
dout = np.random.randn(*x.shape)

dropout_param = {'mode': 'train', 'p': 0.2, 'seed': 123}
out, cache = dropout_forward(x, dropout_param)
dx = dropout_backward(dout, cache)
dx_num = eval_numerical_gradient_array(lambda xx: dropout_forward(xx, dropout_param)[0], x, dout)

# Ошибка должна быть примерно  e-10 или менее
print('Относительная ошибка dx: ', rel_error(dx, dx_num))

## Вопрос 1:
Что произойдет, если мы не разделим значения, передаваемые по обратному пути  на `p` ? 

## Ответ:


# Полносвязанные сети с Dropout
В файле `dlcv / classifiers / fc_net.py` измените  реализацию, чтобы использовать Dropout. Если конструктор сети получает значение, не равное 1 для параметра `dropout`, то сеть должна добавить dropout сразу после каждой нелинейности ReLU. После этого выполните проверку градиента ниже в ячейке.

In [ ]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

for dropout in [1, 0.75, 0.5]:
  print('Выполнение проверки с dropout = ', dropout)
  model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            weight_scale=5e-2, dtype=np.float64,
                            dropout=dropout, seed=123)

  loss, grads = model.loss(X, y)
  print('Начальные потери: ', loss)
  
  # Относительная ошибка должна быть примерно e-6 или менее; отметим, что нормально,
  # если для dropout=1 ошибка W2 будет примерно e-5.

  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
    print('%s относительная ошибка: %.2e' % (name, rel_error(grad_num, grads[name])))
  print()

# Эксперимент с регуляризацией
В качестве эксперимента обучим две двухслойные сети на 500 примерах: одна из них не будет использовать dropout, а другая будет использовать  вероятность сохранения выходов нейронов 0,25. Затем визуализируем зависимости точности обучения и валидации двух сетей с течением времени.

In [ ]:
# Обучим две сети: с dropout и без dropout
np.random.seed(231)
num_train = 500
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

solvers = {}
dropout_choices = [1, 0.25]
for dropout in dropout_choices:
  model = FullyConnectedNet([500], dropout=dropout)
  print('p=',dropout)

  solver = Solver(model, small_data,
                  num_epochs=25, batch_size=100,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 5e-4,
                  },
                  verbose=True, print_every=100)
  solver.train()
  solvers[dropout] = solver

In [ ]:
# Построение графиков точности обучения и валидации для двух моделей

train_accs = []
val_accs = []
for dropout in dropout_choices:
  solver = solvers[dropout]
  train_accs.append(solver.train_acc_history[-1])
  val_accs.append(solver.val_acc_history[-1])

plt.subplot(3, 1, 1)
for dropout in dropout_choices:
  plt.plot(solvers[dropout].train_acc_history, 'o', label='%.2f dropout' % dropout)
plt.title('Точность обучения')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend(ncol=2, loc='lower right')
  
plt.subplot(3, 1, 2)
for dropout in dropout_choices:
  plt.plot(solvers[dropout].val_acc_history, 'o', label='%.2f dropout' % dropout)
plt.title('Валидационная точность')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend(ncol=2, loc='lower right')

plt.gcf().set_size_inches(15, 15)
plt.show()

## Вопрос 2:
Сравните точность валидации и обучения с dropout и без. Как соотносится dropout и регуляризация, что  говорят об этом результаты Вашего эксперимента?

## Ответ:


## Вопрос 3:
Предположим, что мы обучаем глубокую полносвязанную сеть для классификации изображений с dropout после скрытых слоев (параметризуемую вероятностью сохранения активности p). Как нам изменить p,  если мы решим уменьшить размер скрытых слоев (т. е. количество узлов в каждом слое)?

## Ответ:
